#### LIBRERIAS

In [ ]:
import pandas as pd

from datetime import datetime
from datetime import date

import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

import unicodedata
import math
from itertools import cycle
import xlsxwriter
import pyodbc
from datetime import timedelta
import matplotlib

import random
from IPython.display import display, HTML

import sys

import tkinter as tk
from tkinter import ttk
from tkinter import PhotoImage
from PIL import ImageTk, Image
import os


#### FUNCIONES

In [2]:
def calcula_dv(rut_base):
    # CONVERTIR EL RUT BASE A UNA CADENA Y REVERTIRLA
    reversed_digits = list(map(int, str(rut_base)[::-1]))
    # DEFINIR LOS FACTORES (2..7) QUE SE REPITEN EN CICLO
    factors = list(range(2, 8))
    # CALCULAR LA SUMA DE LOS PRODUCTO
    suma = sum(d * f for d, f in zip(reversed_digits, factors * (len(reversed_digits) // len(factors) + 1)))
    # CALCULA DV
    dv = (-suma) % 11
    return 'K' if dv == 10 else str(dv)

def remove_leading_zeros(rut):
    rut_part, dv = rut.split('-')
    rut_part = rut_part.lstrip('0')
    if not rut_part:
        rut_part = '0'
    return f"{rut_part}-{dv}"

def reemplazar_k(identificador):
    return identificador[:-1:] + identificador[-1].upper()

def separar_rut(numero):
    numero_str = str(numero)
    # TOMAR EL ULTIMO DIGITO
    parte_numerica = numero_str[:-1]
    digito_verificador = numero_str[-1]
    # FORMTATEAR GUION
    rut_formateado = f"{parte_numerica}-{digito_verificador}"
    return rut_formateado

In [3]:
### SEÑAL DE APORTES ELEVADOS EN UN DIA PARTICULAR
### FILTRAR TEMA DE LOS DOF O ALERTAS SEGÚN FILTRO ADICIONAL, DE LA CUSTODIA, PROMEDIO DE APORTES, APORTE MAX, ETC
## QRY ADICIONAL AL FILTRADO SOBRE CUSTODIA, ETC; TOMAR APORTE MAX, CUSTODIA EL DÍA DE LA SEÑAL, 

## INCORPORAR DENTRO DEL MANUAL RIESGOS DE LA PLATAFORMA (ESPECIALMENTE RACIONAL)
## AÑADIR CUSTODIA APORTE EN UN DÍA QUE SEA SUMAMENTE ELEVADO
# RELACIONAR MONTO TRANSADO O PROMEDIO DE APORTES PARA FILTRAR SI EL DOF ES NECESARIO DISPARARLO

## AÑADIR ALERTA ADICIONAL QUE HAYA APORTE ELEVADO, (NO SOLO EL PRIMER DIA) Y LEVANTAR LA SEÑAL
# PRUEBAS DE A CUANTO LES GENERARÍA EL DOF


## MUESTRA MÁS HETEROGENEA PARA VER A CUANTOS LES LLEGARIA EL DOF

#### MUESTRA CLIENTES: 

Se toma muestra clientes de Ficha PEP desde Regcheq para desde esta base tomar las operaciones y aportes y rescates.

In [4]:
df_muestra2 = pd.read_excel("M5 - Clientes custodia sobre 40M/Clientes_M5.xlsx")

In [5]:
identificadores = df_muestra2['IDENTIFICADOR'].tolist()
# SEPARAR POR COMAS
m = len(identificadores)
identificadores_str = ','.join([f"'{id}'" for id in identificadores])

#### CONSULTAS:
1. query_APORET: toma los aportes y los retiros
2. query_INFO: toma la info personal
3. query_CUSTODIA: operaciones del cliente

4. query_OPERACIONES ## criterio aportes e inmovilizado sin ningun tipo de operacion
5. query_SALDO_CAJA ## saldo caja primer mes y compararlo a finales, si en ambos esta sobre 10kk sospecha

In [6]:
m = df_muestra2['IDENTIFICADOR'].nunique()

In [7]:
F_i = '2023-06-01'
F = '2024-09-30' 

F_ic = '2024-09-30' 

## tomar menos tiempo custodia
F_ix = '2024-01-01'

#### TOMAR DATOS XLSX

In [8]:
#### TOMA DATOS DESDE EL EXCEL PARA LAS PRUEBAS
df_APORET_detalle = pd.read_excel('M5 - Clientes custodia sobre 40M/aportesM5.xlsx')  
df_APORET_detalle.shape
df_APORET_RESUMEN = df_APORET_detalle.groupby(['IDENTIFICADOR', 'NOMBRE_CLI','CARGO_ABONO', 'TIPO_CAJA'], as_index=False)['MONTO_CLP'].sum()

### DATOS CLIENTES
df_DATOS_clientes = pd.read_excel("M5 - Clientes custodia sobre 40M/datosM5.xlsx")  
## OPERACIONES
df_operaciones = pd.read_excel('M5 - Clientes custodia sobre 40M/operacionesM5.xlsx') 

## CUSTODIA
df_custodia_detalle = pd.read_excel("M5 - Clientes custodia sobre 40M/custodiaM5.xlsx") 
## CAJA
df_saldo_caja = pd.read_excel("M5 - Clientes custodia sobre 40M/cajaM5.xlsx") 

In [9]:
df_saldo_caja['IDENTIFICADOR'] = df_saldo_caja['IDENTIFICADOR'].apply(reemplazar_k)
df_custodia_detalle['IDENTIFICADOR'] = df_custodia_detalle['IDENTIFICADOR'].apply(reemplazar_k)
df_operaciones['IDENTIFICADOR'] = df_operaciones['IDENTIFICADOR'].apply(reemplazar_k)
df_DATOS_clientes['IDENTIFICADOR'] = df_DATOS_clientes['IDENTIFICADOR'].apply(reemplazar_k)
df_APORET_detalle['IDENTIFICADOR'] = df_APORET_detalle['IDENTIFICADOR'].apply(reemplazar_k)

In [10]:
df_dolar_obs = pd.read_excel("dol_observado.xlsx") 

df_operaciones = pd.merge(df_operaciones, df_dolar_obs[['FECHA_OPERACION', 'DOLAR_OBS']], on='FECHA_OPERACION', how='left')
df_operaciones['DOLAR_OBS'] = df_operaciones['DOLAR_OBS'].fillna(' ')

df_operaciones['MONTO_CLP'] = df_operaciones.apply(
    lambda row: row['MONTO'] * row['DOLAR_OBS'] if row['COD_MONEDA_OP'] == 'USD' else row['MONTO'], 
    axis=1
)

In [11]:
#df_operaciones.to_excel("M5 - Clientes custodia sobre 40M/operacionesM5.xlsx")
#df_APORET_detalle.to_excel("M5 - Clientes custodia sobre 40M/aportesM5.xlsx")
#df_DATOS_clientes.to_excel("M5 - Clientes custodia sobre 40M/datosM5.xlsx")
#df_custodia_detalle.to_excel("M5 - Clientes custodia sobre 40M/custodiaM5xlsx")
#df_saldo_caja.to_excel("M5 - Clientes custodia sobre 40M/cajaM5.xlsx")


In [14]:
%store df_saldo_caja
%store df_custodia_detalle
%store df_operaciones
%store df_DATOS_clientes
%store df_APORET_detalle
%store df_muestra2

Stored 'df_saldo_caja' (DataFrame)
Stored 'df_custodia_detalle' (DataFrame)
Stored 'df_operaciones' (DataFrame)
Stored 'df_DATOS_clientes' (DataFrame)
Stored 'df_APORET_detalle' (DataFrame)
Stored 'df_muestra2' (DataFrame)
